In [5]:
# load dataset used in 1.2 into 1.3 - analysis
import pandas as pd

df = pd.read_csv("/Users/frankdong/Documents/Analytics Local/williams-racing-strategies/processed_data/delta-all-circuits.csv")

print(df.head())

   Unnamed: 0  race_id  gp_year               gp_name  gp_round  \
0           0     1029     2019  Brazilian Grand Prix        20   
1           1     1019     2019    British Grand Prix        10   
2           2     1023     2019    Italian Grand Prix        14   
3           3     1015     2019     Monaco Grand Prix         6   
4           4     1019     2019    British Grand Prix        10   

      driver_name constructor constructor_ref  is_williams  start_position  \
0  George Russell    Williams        williams         True              18   
1  George Russell    Williams        williams         True              19   
2  George Russell    Williams        williams         True              14   
3  George Russell    Williams        williams         True              19   
4   Robert Kubica    Williams        williams         True              20   

   final_position  grid_delta gained_or_lost  num_places  
0              12           6         gained           6  
1         